In [1]:
import torch
from replknet import *
from convert_opt_conv import *

C:\Users\shich\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [121]:
# define RepLKNet model
drop_path_rate=0.3
num_classes=1000
use_checkpoint=True
small_kernel_merged=False

model = RepLKNet(large_kernel_sizes=[31,29,27,13], layers=[2,2,18,2], channels=[128,256,512,1024],
                    drop_path_rate=drop_path_rate, small_kernel=5, num_classes=num_classes, use_checkpoint=use_checkpoint,
                    small_kernel_merged=small_kernel_merged, use_sync_bn=False)
model

drop path: Identity()
drop path: DropPath(drop_prob=0.013)
drop path: DropPath(drop_prob=0.026)
drop path: DropPath(drop_prob=0.039)
drop path: DropPath(drop_prob=0.052)
drop path: DropPath(drop_prob=0.065)
drop path: DropPath(drop_prob=0.078)
drop path: DropPath(drop_prob=0.091)
drop path: DropPath(drop_prob=0.104)
drop path: DropPath(drop_prob=0.117)
drop path: DropPath(drop_prob=0.130)
drop path: DropPath(drop_prob=0.143)
drop path: DropPath(drop_prob=0.157)
drop path: DropPath(drop_prob=0.170)
drop path: DropPath(drop_prob=0.183)
drop path: DropPath(drop_prob=0.196)
drop path: DropPath(drop_prob=0.209)
drop path: DropPath(drop_prob=0.222)
drop path: DropPath(drop_prob=0.235)
drop path: DropPath(drop_prob=0.248)
drop path: DropPath(drop_prob=0.261)
drop path: DropPath(drop_prob=0.274)
drop path: DropPath(drop_prob=0.287)
drop path: DropPath(drop_prob=0.300)


RepLKNet(
  (stem): ModuleList(
    (0): Sequential(
      (conv): Conv2d(3, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlinear): ReLU()
    )
    (1): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlinear): ReLU()
    )
    (2): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlinear): ReLU()
    )
    (3): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128, bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlinear): ReLU()
    )
  

In [129]:
stage = None
for i, child in enumerate(model.children()):
    if i == 1:
        stage = child
        break
for i, child in enumerate(stage.children()):
    stage = child
    break
for i, child in enumerate(stage.children()):
    mlist = child
    break
for i, child in enumerate(mlist.children()):
    rblock = child
    break
for i, child in enumerate(rblock.children()):
    seq = child
    break
for i, child in enumerate(seq.children()):
    conv = child
    break
for i, child in enumerate(conv.children()):
    print(i)
def is_lowest(module):
    try:
        first = next(module.children())
    except:
        return True
    return False
type(rblock)

replknet.RepLKBlock

In [138]:
dynamic_convert_opt_conv2d(model, 3, 224, 224)

it is 0th father module <class 'torch.nn.modules.container.ModuleList'>
it is 0th father module <class 'torch.nn.modules.container.Sequential'>
it is a lowest module <class 'torch.nn.modules.conv.Conv2d'> True
it is a lowest module <class 'torch.nn.modules.batchnorm.BatchNorm2d'> True
it is a lowest module <class 'torch.nn.modules.activation.ReLU'> True
it is 1th father module <class 'torch.nn.modules.container.Sequential'>
it is a lowest module <class 'torch.nn.modules.conv.Conv2d'> True
it is a lowest module <class 'torch.nn.modules.batchnorm.BatchNorm2d'> True
it is a lowest module <class 'torch.nn.modules.activation.ReLU'> True
it is 2th father module <class 'torch.nn.modules.container.Sequential'>
it is a lowest module <class 'torch.nn.modules.conv.Conv2d'> True
it is a lowest module <class 'torch.nn.modules.batchnorm.BatchNorm2d'> True
it is a lowest module <class 'torch.nn.modules.activation.ReLU'> True
it is 3th father module <class 'torch.nn.modules.container.Sequential'>
it i

RuntimeError: Given groups=1, weight of size [256, 256, 1, 1], expected input[3, 128, 56, 56] to have 256 channels, but got 128 channels instead

In [136]:
def is_lowest(module):
    try:
        first = next(module.children())
    except:
        return True
    return False

### dynamically decide the better operator for convolution, given parameters
def dynamic_fit_conv(kernel_size, input_width, input_height, input_channel=3, output_channel=2):
    # create simulated input data with batch size of 3
    signal = torch.randn(3, input_channel, input_width, input_height)
    kernel = torch.randn(output_channel, input_channel, kernel_size, kernel_size)
    bias = torch.randn(output_channel)
    
    # initialize candidates: Conv2d, FFTConv2d
    test_torch_conv = torch.nn.Conv2d(input_channel, output_channel, kernel_size, bias=True)
    test_torch_conv.weight = torch.nn.Parameter(kernel)
    test_torch_conv.bias = torch.nn.Parameter(bias)
    
    test_fft_conv = FFTConv2d(input_channel, output_channel, kernel_size, bias=True)
    test_fft_conv.weight = torch.nn.Parameter(kernel)
    test_fft_conv.bias = torch.nn.Parameter(bias)
    
    # run and get average time
    iters = 1
    time0 = time.time()
    for _ in range(iters):
        out = test_torch_conv(signal)
    time1 = time.time()
    
    for _ in range(iters):
        out = test_fft_conv(signal)
    time2 = time.time()
    
    torch_time = (time1 - time0) / iters * 1000
    fft_time = (time2 - time1) / iters * 1000
    
    # 0: torch conv is better; 1: fft conv is better
    return 0 if torch_time < fft_time else 1

def dynamic_convert_opt_conv2d(model, org_in_channels, org_in_width, org_in_height):
    
    cur_in_channels, cur_in_width, cur_in_height = org_in_channels, org_in_width, org_in_height
    
    # traverse through layers and substitute with optimal convolution candidate, w.r.t. key parameters (see function dynamic_fit_conv)
    for i, submodule in enumerate(list(model.children())):
        if not is_lowest(submodule):
            print('it is {}th father module'.format(i), type(submodule))
            cur_in_channels, cur_in_width, cur_in_height = dynamic_convert_opt_conv2d(submodule, cur_in_channels, cur_in_width, cur_in_height)
        else:
            print('it is a lowest module', type(submodule), is_lowest(submodule))
            if isinstance(submodule, torch.nn.Conv2d):
                ### fetch internal data sizes
                ### we must feed correct kernel_size, input_size to the 'dynamic_fit_conv' function
                # print(submodule.kernel_size[0], input_width, input_height, submodule.in_channels, submodule.out_channels)
                best_candidate = dynamic_fit_conv(kernel_size=submodule.kernel_size[0], 
                                            input_width=cur_in_width, input_height=cur_in_height, 
                                            input_channel=submodule.in_channels, output_channel=submodule.out_channels)
                if best_candidate == 1:
                    print('Convert {}th layer to FFTConv2d'.format(i + 1))
                    # org_weight, org_bias = module_output[i].weight, module_output[i].bias
                    submodule = FFTConv2d(in_channels=submodule.in_channels, 
                                                out_channels=submodule.out_channels,
                                                kernel_size=submodule.kernel_size, 
                                                stride=submodule.stride,
                                                padding=submodule.padding,
                                                dilation=submodule.dilation,
                                                groups=submodule.groups,
                                                bias=True if submodule.bias is not None else False)
            tmp = torch.randn(3, cur_in_channels, cur_in_width, cur_in_height)
            cur_in_channels, cur_in_width, cur_in_height = submodule(tmp).size()[-3:]
    return cur_in_channels, cur_in_width, cur_in_height